# Random Forest
Hot nuclei.
Run on Windows / Precision.
Train on 5-class cancer classification.  
Input is the nucleus+RBC rollup, training set, hot nuclei only.    
Run 5-fold cross validation.  
Print the mean accuracy. 

The Image.csv is a copy of the original.  
The RBC_rollup.csv is a copy of the original.  
The Nucleus_rollup.csv summarizes only the high-temperature nuclei.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-11 13:43:16.595881
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K_Hot\\'  # append Output0/ etc
CLASSES=range(0,6)  # use all 6 classes
CLASSES=range(1,6)  # use just 5 classes
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.21'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = cp.get_RBC_rollup()
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-11 13:43:17.922096
2022-06-11 13:43:17.922096
Process class 1
Num WSI in test/train sets: 7 26
Num patches in test/train sets: 2734 10168
2022-06-11 13:43:36.169985
Process class 2
Num WSI in test/train sets: 7 30
Num patches in test/train sets: 2769 11181
2022-06-11 13:43:56.278453
Process class 3
Num WSI in test/train sets: 3 14
Num patches in test/train sets: 1091 5273
2022-06-11 13:44:06.096757
Process class 4
Num WSI in test/train sets: 2 6
Num patches in test/train sets: 813 2384
2022-06-11 13:44:11.055778
Process class 5
Num WSI in test/train sets: 1 3
Num patches in test/train sets: 396 1195


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_BoundingBoxArea_count,Nuc_AreaShape_BoundingBoxArea_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
1209,11.0,424.909091,225.843510,199.0,297.00,345.0,445.00,975.0,11.0,678.181818,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1210,7.0,358.714286,116.318119,202.0,260.00,397.0,454.00,484.0,7.0,556.714286,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1211,18.0,478.388889,229.846532,192.0,305.25,451.5,592.25,1113.0,18.0,817.777778,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1212,5.0,412.200000,178.876214,217.0,258.00,429.0,506.00,651.0,5.0,610.800000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1213,7.0,472.285714,194.381681,214.0,299.50,580.0,628.00,657.0,7.0,734.714286,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,6.0,335.666667,107.265403,233.0,241.75,327.0,417.50,466.0,6.0,607.666667,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1587,4.0,415.000000,163.129805,240.0,302.25,410.5,523.25,599.0,4.0,606.500000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,9.0,520.555556,217.351737,266.0,362.00,437.0,705.00,875.0,9.0,794.666667,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-11 13:44:14.818200
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Cross-validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-11 13:44:15.445372
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.3min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV] END .................................................... total time= 2.3min
[CV] END .................................................... total time= 2.3min
[CV] END .................................................... total time= 2.5min
[CV] END .................................................... total time= 2.4min
[0.65648086 0.6536266  0.64875756 0.64875756 0.63308144]
mean 0.6481 +/- 0.0081


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.9min finished


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_model(),MODELS_DIR)

2022-06-11 13:56:08.577059


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.21']